In [1]:
!pip install -q gradio >> out.txt
!pip install presidio_analyzer >> out.txt
!pip install presidio_anonymizer >> out.txt
!python -m spacy download en_core_web_lg >> out.txt

2023-10-16 03:33:21.527135: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 03:33:22.872963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import gradio as gr
import pandas as pd
from gpt_assistant import Assistant
from privacy import Privacy

In [3]:
def privacy_ner(user_prompt, parameters):
  assistant = Assistant()
  privacy_obj = Privacy()
  analyzer_results = privacy_obj.entity_detector(user_prompt)
  masked_text, mask_dict = privacy_obj.mask_entities(user_prompt, analyzer_results, parameters)
  reply_masked = assistant.completion(prompt=masked_text)
  reply_unmasked = privacy_obj.unmask_entities(reply_masked, mask_dict)
  return f"""
Masked Response from LLM:

  {reply_masked}


UnMasked Response from LLM:

  {reply_unmasked}


Masked User Prompt:

  {masked_text}
  """

In [4]:
data = [
    ['CREDIT_CARD', 'A credit card number is between 12 to 19 digits.'],
    ['CRYPTO', 'A Crypto wallet number. Currently only Bitcoin address is supported Pattern match'],
    ['EMAIL_ADDRESS', 'An email address identifies an email box to which email messages are delivered Pattern match'],
    ['IBAN_CODE', 'The International Bank Account Number (IBAN) is an internationally agreed system of identifying bank accounts across national borders to facilitate the communication and processing of cross border transactions with a reduced risk of transcription errors.'],
    ['IP_ADDRESS', 'An Internet Protocol (IP) address (either IPv4 or IPv6).'],
    ['NRP', 'A person’s Nationality, religious or political group.'],
    ['LOCATION', 'Name of politically or geographically defined location (cities, provinces, countries, international regions, bodies of water, mountains Custom logic and context'],
    ['PERSON', 'A full person name, which can include first names, middle names or initials, and last names.'],
    ['PHONE_NUMBER', 'A telephone number'],
    ['MEDICAL_LICENSE', 'Common medical license numbers.']
]
df = pd.DataFrame(data, columns=["PII", "Description"])

In [5]:
for index, row in df.iterrows():
  print(row)

PII                                                 CREDIT_CARD
Description    A credit card number is between 12 to 19 digits.
Name: 0, dtype: object
PII                                                       CRYPTO
Description    A Crypto wallet number. Currently only Bitcoin...
Name: 1, dtype: object
PII                                                EMAIL_ADDRESS
Description    An email address identifies an email box to wh...
Name: 2, dtype: object
PII                                                    IBAN_CODE
Description    The International Bank Account Number (IBAN) i...
Name: 3, dtype: object
PII                                                   IP_ADDRESS
Description    An Internet Protocol (IP) address (either IPv4...
Name: 4, dtype: object
PII                                                          NRP
Description    A person’s Nationality, religious or political...
Name: 5, dtype: object
PII                                                     LOCATION
Description    Name

In [8]:
demo = gr.Interface(
    privacy_ner,
    [
        gr.inputs.Textbox(lines=5, label="Input Text"),
        gr.CheckboxGroup(['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'NRP', 'LOCATION', 'PERSON', 'PHONE_NUMBER', 'MEDICAL_LICENSE'], label="PII to be masked", info="""
CREDIT_CARD : A credit card number is between 12 to 19 digits.

CRYPTO : A Crypto wallet number. Currently only Bitcoin address is supported.

EMAIL_ADDRESS : An email address identifies an email box to which email messages are delivered.

IBAN_CODE : The International Bank Account Number (IBAN) is an internationally agreed system of identifying bank accounts across national borders to facilitate the communication and processing of cross border transactions with a reduced risk of transcription errors.

IP_ADDRESS : An Internet Protocol (IP) address (either IPv4 or IPv6).

NRP : A person’s Nationality, religious or political group.

LOCATION : Name of politically or geographically defined location (cities, provinces, countries, international regions, bodies of water, mountains.

PERSON : A full person name, which can include first names, middle names or initials, and last names.

PHONE_NUMBER : A telephone number.

MEDICAL_LICENSE : Common medical license numbers.
        """),
    ],
    "text",
)

In [9]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://49eafc2b63542a7991.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://49eafc2b63542a7991.gradio.live
